# Decomposing Qiskit circuits

In this notebook, we show how Qiskit circuits can be converted into Perceval circuits. To do so, we take the example of a simple gate-based circuit (a circuit producing GHZ states) and we show the translation to a linear optical circuit. We also show the equivalence between the two circuits.

As usual we start by imported the useful libraries. Note that this notebook requires the installation of Qiskit (which can be easiliy done with `pip install qiskit`).

In [1]:
import perceval as pcvl
import perceval.components.base_components as comp
from perceval.components.core_catalog import catalog
from perceval.converters import QiskitConverter

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

## GHZ State generation in Qiskit

We first define the circuit generating GHZ states of 3 qubits with Qiskit. To do so, we first act with a Hadamard gate on qubit 0 to put in superposition of state $|0\rangle$ and $|1\rangle$. Then we perform two CNOT gates using qubit 0 as control and qubits 1 and 2 as targets.

In [2]:
# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(3)

# Add a H gate on qubit 0
circuit.h(0)

# Add CX (CNOT) gates on control qubit 0 and target qubits 1 and 2
circuit.cx(0, 1)
circuit.cx(0, 2)

# Draw the circuit
circuit.draw()

┌───┐          
q_0: ┤ H ├──■────■──
     └───┘┌─┴─┐  │  
q_1: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘

We display the final state when starting from the input state $|000\rangle$.

In [3]:
# Set the initial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2**3)

# Evolve the state by the quantum circuit
state = state.evolve(circuit)

#draw using latex
state.draw('latex')

<IPython.core.display.Latex object>

## Conversion to Perceval

With the use of `QiskitConverter`, we can transform the Qiskit circuit into a Perceval circuit. It uses 2 modes per qubit and additional modes for ancillary photons to perform deterministically two-qubit gates. Below the first six modes correspond to the three logical qubits (see the 'Spatial Modes encoding' paragraph in the 'Basics' section of the documentation) of the gate-based circuit above.

The other modes are used to successfully implement two-qubit gates via heralding or post-selection. Heralding employs [4 ancillary modes](https://doi.org/10.1073/pnas.1018839108) while post-selection employs [2 ancillary modes](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.65.062324). With the option `heralded=True` in the method `.convert` on a `QiskitConverter` object, every CNOT but the last is implemented with a heralding scheme. Here it means that it would add $4+2$ ancillary modes. The option `heralded=True` only implements post-selected CNOTs. Here it would mean $2+2$ ancillary modes. A cautionary warning: post-selected CNOTs do not compose generally.

In [4]:
qiskit_converter = QiskitConverter(catalog)
quantum_processor = qiskit_converter.convert(circuit, heralded=True)
pcvl.pdisplay(quantum_processor, recursive=True)

With this converted circuit, we can now check that the resulting state is the same as before the conversion.

In [5]:
simulator_backend = pcvl.BackendFactory().get_backend("Naive")

_, output_distribution = quantum_processor.run(simulator_backend)
pcvl.pdisplay(output_distribution, precision=1e-2)

state,probability
"|1,0,1,0,1,0>",1/2
"|0,1,0,1,0,1>",1/2
"|1,0,0,1,1,0>",0
"|1,0,1,0,0,1>",0
"|0,1,0,1,1,0>",0
"|0,1,1,0,0,1>",0
"|1,0,0,1,0,1>",0
"|0,1,1,0,1,0>",0


This circuit can now be converted using a general interferometer decomposition so it can be implemented on a generic photonic chip.

In [6]:
u = quantum_processor.circuit.compute_unitary(use_symbolic=False)

In [7]:
ub = (pcvl.Circuit(2)
      // comp.GenericBS(theta=pcvl.Parameter("theta"))
      // (0, comp.PS(phi=pcvl.Parameter("φ_a"))))

pc_norm = pcvl.Circuit.decomposition(u, ub, shape="triangle")
pcvl.pdisplay(pc_norm, compact=True, render_size=0.5)